In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from numpy.random import seed
import sklearn.model_selection as ms

In [2]:
seed(1)
tf.random.set_seed(2)

In [3]:
aapl_dataset_fp = 'dataset/ta_zm_1.5y.csv'
class_names = ['LOW', 'HIGH']

In [4]:
aapl_dataset = pd.read_csv(aapl_dataset_fp)
aapl_dataset

,roc,rsi,stoch,stochrsi,stochrsi_d,stochrsi_k,macd,macd_diff,macd_signal,tsi,tsi_signal,roc_close_ema_w8,roc_ema_w8_ema_w20,roc_close_ema_w20,roc_close_sma_w9,roc_sma_w9_sma_w26,roc_close_sma_w26,trend
0,-13.957401,47.483400,15.511565,0.085089,0.127451,0.154245,1.871502,-1.975828,3.847330,10.699120,21.190870,-3.863964,-0.198966,-4.055242,-8.259399,5.184573,-3.503041,HIGH
1,-15.116167,45.682000,8.374584,0.006186,0.128629,0.088533,1.251217,-2.076890,3.328107,8.226750,19.338853,-4.553028,-0.917577,-5.428827,-8.915906,3.494144,-5.733297,HIGH
2,-7.807567,51.744656,30.321781,0.307038,0.125183,0.132771,1.175368,-1.722192,2.897559,7.711835,17.677850,1.094561,-0.652270,0.435151,-2.433665,1.992802,-0.489360,HIGH
3,-7.788039,53.254958,36.221121,0.381958,0.151011,0.231727,1.216621,-1.344750,2.561372,7.702633,16.252819,2.081980,-0.249949,1.826827,-0.376165,0.956423,0.576659,HIGH
4,-11.358528,51.885253,31.600679,0.314013,0.232945,0.334336,1.145734,-1.132511,2.278244,7.317638,14.976365,0.650276,-0.119978,0.529517,-0.856250,-0.272951,-1.126864,HIGH
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
497,15.748719,65.806793,82.738461,0.805088,0.897581,0.931541,13.768648,3.252852,10.515796,29.676787,19.187641,1.759502,4.375635,6.212127,4.611606,4.507842,9.327332,HIGH
498,11.502974,67.858948,91.498995,0.909955,0.920661,0.905014,14.166713,2.920734,11.245979,30.709396,20.833606,2.507855,4.357130,6.974256,4.869860,4.958138,10.069452,HIGH
499,9.998252,68.709124,90.800798,0.932352,0.906340,0.882465,14.498228,2.601799,11.896429,31.804263,22.400842,2.416827,4.325405,6.846769,4.583799,5.091761,9.908956,HIGH
500,9.418034,65.802484,82.259907,0.716737,0.880164,0.853014,14.311940,1.932409,12.379531,31.897729,23.757540,1.132166,4.084176,5.262581,2.931757,5.010368,8.089017,HIGH


In [5]:
feature_names = aapl_dataset.columns[:-1]
label_name = aapl_dataset.columns[-1]

print("Features: {}".format(feature_names))
print("Label: {}".format(label_name))

Features: Index(['roc', 'rsi', 'stoch', 'stochrsi', 'stochrsi_d', 'stochrsi_k', 'macd',
       'macd_diff', 'macd_signal', 'tsi', 'tsi_signal', 'roc_close_ema_w8',
       'roc_ema_w8_ema_w20', 'roc_close_ema_w20', 'roc_close_sma_w9',
       'roc_sma_w9_sma_w26', 'roc_close_sma_w26'],
      dtype='object')
Label: trend


In [6]:
aapl_dataset[label_name] = aapl_dataset[label_name].map({'LOW': 0, 'HIGH': 1})
features = aapl_dataset.copy()
labels = features.pop(label_name)
features = tf.keras.utils.normalize(features, axis=-2, order=2)

features

,roc,rsi,stoch,stochrsi,stochrsi_d,stochrsi_k,macd,macd_diff,macd_signal,tsi,tsi_signal,roc_close_ema_w8,roc_ema_w8_ema_w20,roc_close_ema_w20,roc_close_sma_w9,roc_sma_w9_sma_w26,roc_close_sma_w26
0,-0.042292,0.038604,0.011230,0.006116,0.009626,0.011476,0.006070,-0.024771,0.013079,0.020564,0.043421,-0.034097,-0.001865,-0.019885,-0.046442,0.042485,-0.013033
1,-0.045803,0.037139,0.006063,0.000445,0.009715,0.006587,0.004058,-0.026038,0.011314,0.015812,0.039626,-0.040178,-0.008603,-0.026620,-0.050133,0.028633,-0.021331
2,-0.023657,0.042068,0.021953,0.022070,0.009455,0.009878,0.003812,-0.021591,0.009850,0.014822,0.036223,0.009659,-0.006115,0.002134,-0.013684,0.016330,-0.001821
3,-0.023598,0.043296,0.026224,0.027455,0.011405,0.017240,0.003946,-0.016859,0.008707,0.014805,0.033303,0.018372,-0.002343,0.008958,-0.002115,0.007837,0.002145
4,-0.034417,0.042183,0.022879,0.022571,0.017593,0.024874,0.003716,-0.014198,0.007745,0.014065,0.030687,0.005738,-0.001125,0.002596,-0.004815,-0.002237,-0.004192
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
497,0.047719,0.053501,0.059902,0.057869,0.067790,0.069305,0.044658,0.040780,0.035747,0.057039,0.039316,0.015527,0.041023,0.030461,0.025931,0.036940,0.034702
498,0.034855,0.055169,0.066245,0.065407,0.069534,0.067331,0.045949,0.036617,0.038229,0.059024,0.042689,0.022131,0.040850,0.034198,0.027383,0.040630,0.037463
499,0.030295,0.055860,0.065739,0.067017,0.068452,0.065654,0.047024,0.032618,0.040441,0.061129,0.045900,0.021327,0.040552,0.033573,0.025774,0.041725,0.036866
500,0.028537,0.053497,0.059556,0.051519,0.066475,0.063463,0.046420,0.024226,0.042083,0.061308,0.048680,0.009991,0.038291,0.025805,0.016485,0.041058,0.030095


In [7]:
def create_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(20, activation=tf.nn.relu, input_shape=(17,)),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(20, activation=tf.nn.relu),
        tf.keras.layers.Dropout(0.1),
        tf.keras.layers.Dense(20, activation=tf.nn.relu),
        tf.keras.layers.Dense(2)
    ])

    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
        loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],
    )

    return model

In [8]:
x = features.values
y = labels.values

for train_index, test_index in ms.KFold(10).split(x):
    x_train, x_test = x[train_index], x[test_index]
    y_train, y_test = y[train_index], y[test_index]
  
    model = create_model()
    model.fit(x_train, y_train,epochs=100, verbose=0)
  
    print('Model evaluation ',model.evaluate(x_test,y_test))

2/2 [==============================] - 0s 2ms/step - loss: 1.8469 - sparse_categorical_accuracy: 0.1569
Model evaluation  [1.8468698263168335, 0.1568627506494522]
2/2 [==============================] - 0s 3ms/step - loss: 0.6090 - sparse_categorical_accuracy: 0.7255
Model evaluation  [0.6090134382247925, 0.7254902124404907]
2/2 [==============================] - 0s 2ms/step - loss: 0.8542 - sparse_categorical_accuracy: 0.4000
Model evaluation  [0.8541576862335205, 0.4000000059604645]
2/2 [==============================] - 0s 3ms/step - loss: 0.1538 - sparse_categorical_accuracy: 0.9800
Model evaluation  [0.1538173109292984, 0.9800000190734863]
2/2 [==============================] - 0s 2ms/step - loss: 0.3738 - sparse_categorical_accuracy: 0.9400
Model evaluation  [0.37379348278045654, 0.9399999976158142]
2/2 [==============================] - 0s 2ms/step - loss: 1.0197 - sparse_categorical_accuracy: 0.3000
Model evaluation  [1.0196599960327148, 0.30000001192092896]
2/2 [===============